# 读写文件

到目前为止，我们讨论了如何处理数据，
以及如何构建、训练和测试深度学习模型。
然而，有时我们希望保存训练的模型，
以备将来在各种环境中使用（比如在部署中进行预测）。
此外，当运行一个耗时较长的训练过程时，
最佳的做法是定期保存中间结果，
以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。
因此，现在是时候学习如何加载和存储权重向量和整个模型了。

## (**加载和保存张量**)

对于单个张量，我们可以直接调用`load`和`save`函数分别读写它们。
这两个函数都要求我们提供一个名称，`save`要求将要保存的变量作为输入。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

我们现在可以将存储在文件中的数据读回内存。

In [2]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

我们可以[**存储一个张量列表，然后把它们读回内存。**]

In [3]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们甚至可以(**写入或读取从字符串映射到张量的字典**)。
当我们要读取或写入模型中的所有权重时，这很方便。

In [4]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## [**加载和保存模型参数**]

保存单个权重向量（或其他张量）确实有用，
但是如果我们想保存整个模型，并在以后加载它们，
单独保存每个向量则会变得很麻烦。
毕竟，我们可能有数百个参数散布在各处。
因此，深度学习框架提供了内置函数来保存和加载整个网络。
**需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。**
例如，如果我们有一个3层多层感知机，我们需要单独指定架构。
因为模型本身可以包含任意代码，所以模型本身难以序列化。
因此，为了恢复模型，我们需要用代码生成架构，
然后从磁盘加载参数。
让我们从熟悉的多层感知机开始尝试一下。

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
X, Y

(tensor([[-0.6704, -0.0873, -1.5169,  1.1367, -1.8229,  0.1768, -0.4015,  0.3436,
          -1.3141,  0.2494,  0.9123, -2.3178,  0.3741, -1.3778,  1.0040, -0.4411,
           0.2945, -1.9843, -0.8758, -0.8836],
         [ 0.2729, -0.2969, -1.1340,  0.6454,  0.2003, -0.6047,  0.2793,  0.5907,
           0.5478,  0.1890, -1.4201, -0.4658, -0.8422,  0.4994,  0.1606, -0.3161,
           0.4686,  0.0990, -0.5713, -1.2930]]),
 tensor([[-0.2349,  0.3141,  0.0643, -0.5557, -0.0561,  0.0798,  0.0691,  0.5871,
          -0.3742,  0.3844],
         [-0.0560,  0.2043, -0.0851, -0.0547,  0.0564,  0.1259, -0.0427,  0.1011,
           0.0191,  0.0592]], grad_fn=<AddmmBackward0>))

接下来，我们[**将模型的参数存储在一个叫做“mlp.params”的文件中。**]

In [6]:
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[ 0.2078,  0.0817, -0.0243,  ...,  0.1357,  0.0890, -0.0911],
                      [-0.0420,  0.0313, -0.0831,  ..., -0.2114, -0.2186,  0.1301],
                      [-0.1529, -0.1986,  0.0133,  ...,  0.0553, -0.1895,  0.1850],
                      ...,
                      [-0.0754,  0.2184,  0.0838,  ...,  0.1210, -0.2118, -0.0100],
                      [-0.1751,  0.0602, -0.0281,  ...,  0.1957,  0.0483,  0.1417],
                      [-0.0989, -0.0331, -0.0775,  ...,  0.0638,  0.1482,  0.1427]])),
             ('hidden.bias',
              tensor([-2.3199e-02, -2.0725e-01, -1.9671e-01,  4.2320e-02,  2.1258e-01,
                       1.8176e-01, -2.1460e-01,  1.8229e-01,  1.2728e-01,  2.1731e-01,
                      -1.2655e-01, -1.6592e-02,  9.1806e-03, -1.6071e-01,  2.8347e-02,
                       7.3912e-02,  1.9067e-01,  5.5350e-02,  7.9832e-02,  5.6686e-02,
                      -2.2262e-01, -2.1531e-01, -1.1968e-0

In [7]:
torch.save(net.state_dict(), 'mlp.params')

为了恢复模型，我们[**实例化了原始多层感知机模型的一个备份。**]
这里我们不需要随机初始化模型参数，而是(**直接读取文件中存储的参数。**)

In [8]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

由于两个实例具有相同的模型参数，在输入相同的`X`时，
两个实例的计算结果应该相同。
让我们来验证一下。

注：eval评估模式下dropout会被关掉

In [9]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 小结

* `save`和`load`函数可用于张量对象的文件读写。
* 我们可以通过参数字典保存和加载网络的全部参数。
* 保存架构必须在代码中完成，而不是在参数中完成。

## 练习

1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？
1. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？
1. 如何同时保存网络架构和参数？需要对架构加上什么限制？

### 练习一

1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？

**解答：**

&emsp;&emsp;1.便于模型恢复和微调：存储模型参数可以在训练中断后继续训练，或者在新的数据集上进行微调，而无需从头开始训练。

&emsp;&emsp;2.节省存储空间：相比于保存完整的模型结构，保存模型参数通常占用更少的存储空间，这在处理大型模型或存储空间受限的情况下尤为重要。

&emsp;&emsp;3.便于共享和复现：存储模型参数可以方便地共享和复现已经训练好的模型，其他人可以直接加载这些参数并使用它们进行预测或进一步训练。

&emsp;&emsp;4.便于调试和分析：通过检查和分析模型参数，可以更容易地诊断模型中存在的问题，并对模型进行调整和优化。

&emsp;&emsp;下面将以一个简单的例子具体说明。

&emsp;&emsp;1. 便于模型恢复和微调：假设我们有一个简单的神经网络模型，我们可以使用 PyTorch 保存和加载模型参数：

In [10]:
import torch
import torch.nn as nn

# 定义模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 2)

    def forward(self, x):
        return self.fc(x)

model = SimpleModel()

# 保存模型参数
torch.save(model.state_dict(), 'model_parameters.pth')

# 加载模型参数进行微调
model.load_state_dict(torch.load('model_parameters.pth'))
# 继续训练或进行微调...

<All keys matched successfully>

&emsp;&emsp;2. 节省存储空间：在上述示例中，通过使用`torch.save(model.state_dict(), 'model_parameters.pth')`只保存模型参数，而不是整个模型，可以节省存储空间。

&emsp;&emsp;3. 便于共享和复现：保存模型参数后，可以将 `model_parameters.pth` 文件与他人共享。他们可以使用以下代码加载参数并复现模型：

In [11]:
model = SimpleModel()
model.load_state_dict(torch.load('model_parameters.pth'))
# 使用模型进行预测或进一步训练...

<All keys matched successfully>

&emsp;&emsp;4. 便于调试和分析：通过检查模型参数，可以对模型进行调试和分析。例如，可以打印出模型权重和偏置：

In [12]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        print(param.data)

fc.weight
tensor([[-1.2774e-01,  2.1785e-01, -3.8828e-06,  2.9046e-01,  1.6467e-01,
         -3.0214e-02, -1.3911e-02,  2.2464e-01, -3.1618e-01, -1.2282e-01],
        [ 2.1680e-01,  5.9495e-02, -1.3927e-03, -2.9732e-01, -2.0234e-01,
         -1.0656e-01,  8.8981e-02,  6.3178e-02,  5.5092e-02, -2.4218e-02]])
fc.bias
tensor([0.0937, 0.0920])


### 练习二

2. 假设我们只想复用网络的一部分，以将其合并到不同的网络架构中。比如想在一个新的网络中使用之前网络的前两层，该怎么做？

**解答：**

&emsp;&emsp;使用保存模型某层参数的办法，保存网络的前两层，然后再加载到新的网络中使用。

In [13]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):
    """定义 MLP 类。"""
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 定义隐藏层，输入尺寸为20，输出尺寸为256。
        self.output = nn.Linear(256, 10) # 定义输出层，输入尺寸为256，输出尺寸为10

    def forward(self, x):
        """定义前向传播函数。"""
        # 使用 ReLU 激活函数，计算隐藏层和输出层的输出
        return self.output(F.relu(self.hidden(x)))

class MLP_new(nn.Module):
    """定义 MLP_new 类。"""
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 定义隐藏层，输入尺寸为 20，输出尺寸为 256。
        self.output = nn.Linear(256, 10)  # 定义输出层，输入尺寸为 256，输出尺寸为 10。

    def forward(self, x):
        """定义前向传播函数。"""
        # 使用 ReLU 激活函数，计算隐藏层和输出层的输出
        return self.output(F.relu(self.hidden(x)))

net = MLP() # 创建 MLP 的实例
# 将隐藏层的参数保存到文件中。
torch.save(net.hidden.state_dict(), 'mlp.hidden.params')

clone = MLP_new() # 创建另一个 MLP 的实例。
# 加载已保存的参数到克隆实例的隐藏层中。
clone.hidden.load_state_dict(torch.load('mlp.hidden.params'))

# 比较两个 MLP 示例的隐藏层权重是否相等，并输出结果
print(clone.hidden.weight == net.hidden.weight)

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])


### 练习三

3. 如何同时保存网络架构和参数？需要对架构加上什么限制？

&emsp;&emsp;在PyTorch中，可以使用`torch.save()`函数同时保存网络架构和参数。为了保存网络架构，需要将模型的结构定义在一个Python类中，并将该实例化为模型对象。此外，必须确保该类的构造函数不包含任何随机性质的操作，例如`dropout`层的随机丢弃率应该是固定的。

In [14]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):
    """定义 MLP 类。"""
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 定义隐藏层，输入尺寸为 20，输出尺寸为 256。
        self.output = nn.Linear(256, 10) # 定义输出层，输入磁村为 256，输出尺寸为 10。

    def forward(self, x):
        """定义前向传播函数。"""
        # 使用 ReLU 激活函数，计算隐藏层和输出层的输出
        return self.output(F.relu(self.hidden(x)))

net = MLP()

# 存储模型
torch.save(net.state_dict(), 'model.pt')

# 导入模型
model = torch.load('model.pt')
model

OrderedDict([('hidden.weight',
              tensor([[ 0.1919,  0.1739,  0.1791,  ...,  0.0264,  0.1204, -0.0649],
                      [ 0.0598,  0.2070,  0.1452,  ..., -0.0704,  0.1981, -0.0343],
                      [-0.1819,  0.1304, -0.1353,  ...,  0.1365,  0.0960,  0.1419],
                      ...,
                      [-0.0773, -0.2093, -0.0257,  ..., -0.0257, -0.1069, -0.1072],
                      [ 0.0205,  0.0114,  0.1147,  ..., -0.0531, -0.1401,  0.0911],
                      [ 0.0822, -0.1318,  0.1897,  ...,  0.1369, -0.0851, -0.1400]])),
             ('hidden.bias',
              tensor([ 0.2213,  0.1411,  0.1608,  0.1866,  0.1896, -0.1128, -0.0291,  0.1449,
                       0.1427,  0.1917, -0.1950,  0.0390,  0.1072, -0.1042, -0.1064,  0.1581,
                       0.1348,  0.0556,  0.0827,  0.0391, -0.0792,  0.1127, -0.0209,  0.0501,
                       0.0423, -0.0273, -0.1840,  0.1553, -0.1338,  0.0065, -0.1273,  0.1443,
                       0.0163,